In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyreadstat
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [2]:
# calling biomarkers
bloodbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk1_20231117.csv"
bloodbiomk1 = pd.read_csv(bloodbiomk1_file_path)
bloodbiomk1 = pd.DataFrame(bloodbiomk1)

bloodbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\bloodbiomk2_20231117.csv"
bloodbiomk2 = pd.read_csv(bloodbiomk2_file_path)
bloodbiomk2 = pd.DataFrame(bloodbiomk2)

csfbiomk1_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk1_20231117.csv"
csfbiomk1 = pd.read_csv(csfbiomk1_file_path)
csfbiomk1 = pd.DataFrame(csfbiomk1)

csfbiomk2_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_20231117.csv"
csfbiomk2 = pd.read_csv(csfbiomk2_file_path)
csfbiomk2 = pd.DataFrame(csfbiomk2)

csfbiomk3_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk3_20231117.csv"
csfbiomk3 = pd.read_csv(csfbiomk3_file_path)
csfbiomk3 = pd.DataFrame(csfbiomk3)




In [3]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'csf_biomk2_to_neuropsych'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)

# save_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\blood_biomk1_neuropsych" # Update this path to your desired folder


In [4]:
# Check for null values in each column
csfbiomk2_nulls = csfbiomk2.isnull().sum()

csfbiomk2_nulls_df = pd.DataFrame({
    'Column': csfbiomk2_nulls.index,
    'Number of Nulls': csfbiomk2_nulls.values
})

csfbiomk2_nulls_df_transposed = csfbiomk2_nulls_df.T

print("Null values in csfbiomk2:")
print(csfbiomk2_nulls_df)
csfbiomk2_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\csfbiomk2_nulls.csv", index=False)


Null values in csfbiomk2:
                   Column  Number of Nulls
0              subject_id                0
1                   visit                0
2             checkin_bin                0
3             exposurebin                0
4              age_decade                0
..                    ...              ...
127           c_VEGF_A_cv               57
128              c_VEGF_A               57
129             c_asyn_cv               85
130                c_asyn               85
131  dxcte_asyn_c_flag_dv               60

[132 rows x 2 columns]


In [5]:
csfbiomk2 = csfbiomk2.drop(columns=csfbiomk2.loc[:,'subject_id':'chiiyrs_pg'].columns)
csfbiomk2 = csfbiomk2.drop(columns='dxcte_albumin_csf_flag_dv') # This is not for this dataset unless you find a column where there are too many nulls

csfbiomk2.head()

,Qalb,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,c_IL_17D,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,7.852013,30.721,7.79,0.0898,1.09,16.2,9.24,0.0477,0.646,384.0,...,0.789,1.07,0.653,2.4,0.159,2.22,2.09,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
csfbiomk2.head(10)

,Qalb,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,c_IL_17D,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,7.852013,30.721,7.79,0.0898,1.09,16.2,9.240,0.0477,0.646,384.0,...,0.789,1.07,0.653,2.400,0.159,2.22,2.09,NaN,NaN,1.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.743853,27.203,4.51,0.0715,1.84,15.2,0.496,0.0326,1.160,554.0,...,1.130,2.51,0.911,0.599,0.193,2.94,1.93,0.595,486.0,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Calculate the mean for each column, ignoring NaNs
mean_values = csfbiomk2.mean()

csfbiomk2 = csfbiomk2.fillna(mean_values)

csfbiomk2.head()

,Qalb,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,c_IL_17D,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,7.852013,30.721000,7.79000,0.089800,1.090000,16.200000,9.240000,0.04770,0.646000,384.000000,...,0.789000,1.070000,0.653000,2.400000,0.159000,2.220000,2.090000,2.549289,336.278146,1.000000
1,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
2,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
3,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
4,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045


In [8]:
# calling Neuropsychiatric Measurements

neuropsych_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\neuropsych_m_p_20231117.csv"
neuropsych = pd.read_csv(neuropsych_file_path)
neuropsych = pd.DataFrame(neuropsych)



In [9]:
# Check for null values in each column
neuropsych_nulls = neuropsych.isnull().sum()

neuropsych_nulls_df = pd.DataFrame({
    'Column': neuropsych_nulls.index,
    'Number of Nulls': neuropsych_nulls.values
})

neuropsych_nulls_df_transposed = neuropsych_nulls_df.T

print("Null values in neuropsych:")
print(neuropsych_nulls_df)
neuropsych_nulls_df_transposed.to_csv(r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\neuropsych_nulls.csv", index=False)


Null values in neuropsych:
                   Column  Number of Nulls
0              subject_id                0
1                   visit                0
2             checkin_bin                0
3             exposurebin                0
4              age_decade                0
..                    ...              ...
96          explosivity_z               58
97       emo_dyscontrol_z               58
98          impulsivity_z               58
99   affective_lability_z               58
100             nbd_tot_z               58

[101 rows x 2 columns]


In [10]:
neuropsych.head()

,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,explosivity,emo_dyscontrol,impulsivity,affective_lability,nbd_tot,explosivity_z,emo_dyscontrol_z,impulsivity_z,affective_lability_z,nbd_tot_z
0,1001,1,2,1,1,5,16.0,7.0,4335.4,2167.7,...,0.26667,0.33333,0.45455,0.20000,31.3636,-1.01754,-1.18960,-0.28658,-0.88620,-1.05786
1,1002,1,2,1,1,5,15.0,14.0,10363.1,5708.1,...,0.47500,0.75000,0.75758,0.86667,71.2311,0.37271,0.88637,1.77238,2.78227,1.77989
2,1003,1,2,1,1,5,18.0,12.0,6685.4,4863.9,...,0.61667,0.91667,0.59848,0.46667,64.9621,1.31809,1.71676,0.69143,0.58118,1.33367
3,1004,1,1,1,2,5,16.0,16.0,7701.2,6448.9,...,0.33333,0.41667,0.33333,0.20000,32.0833,-0.57266,-0.77441,-1.11016,-0.88620,-1.00663
4,1005,1,3,0,2,5,21.0,NaN,NaN,NaN,...,0.26667,0.50000,0.31061,0.20000,31.9318,NaN,NaN,NaN,NaN,NaN


In [11]:
# Calculate the mean for each column, ignoring NaNs
mean_values = neuropsych.mean()

neuropsych = neuropsych.fillna(mean_values)

neuropsych.head()


,subject_id,visit,checkin_bin,exposurebin,age_decade,racecat_combined,eduyears,totyr_foot,chiiseas_pf,chiiyrs_pf,...,explosivity,emo_dyscontrol,impulsivity,affective_lability,nbd_tot,explosivity_z,emo_dyscontrol_z,impulsivity_z,affective_lability_z,nbd_tot_z
0,1001,1,2,1,1,5,16.0,7.000000,4335.40,2167.700000,...,0.26667,0.33333,0.45455,0.20000,31.3636,-1.017540e+00,-1.189600e+00,-2.865800e-01,-8.862000e-01,-1.057860e+00
1,1002,1,2,1,1,5,15.0,14.000000,10363.10,5708.100000,...,0.47500,0.75000,0.75758,0.86667,71.2311,3.727100e-01,8.863700e-01,1.772380e+00,2.782270e+00,1.779890e+00
2,1003,1,2,1,1,5,18.0,12.000000,6685.40,4863.900000,...,0.61667,0.91667,0.59848,0.46667,64.9621,1.318090e+00,1.716760e+00,6.914300e-01,5.811800e-01,1.333670e+00
3,1004,1,1,1,2,5,16.0,16.000000,7701.20,6448.900000,...,0.33333,0.41667,0.33333,0.20000,32.0833,-5.726600e-01,-7.744100e-01,-1.110160e+00,-8.862000e-01,-1.006630e+00
4,1005,1,3,0,2,5,21.0,15.852778,10914.89,8229.102222,...,0.26667,0.50000,0.31061,0.20000,31.9318,5.056180e-07,5.056180e-07,-1.123596e-07,5.617978e-07,5.617978e-08


In [12]:
neuropsych_nulls = neuropsych.isnull().sum()
neuropsych_nulls

subject_id              0
visit                   0
checkin_bin             0
exposurebin             0
age_decade              0
                       ..
explosivity_z           0
emo_dyscontrol_z        0
impulsivity_z           0
affective_lability_z    0
nbd_tot_z               0
Length: 101, dtype: int64

In [13]:
neuropsych = neuropsych[['bistot','abis_attention','abis_motor','abis_nonplanning','bhstot','bdhi_total', 'cnstot', 'bditot', 'baitot', 'pcltot', 
'BGLHA_Childhood_Total', 'BGLHA_Adolescence_Total', 'BGLHA_Adulthood_Total', 'nbd_tot' ]]
neuropsych.head()

,bistot,abis_attention,abis_motor,abis_nonplanning,bhstot,bdhi_total,cnstot,bditot,baitot,pcltot,BGLHA_Childhood_Total,BGLHA_Adolescence_Total,BGLHA_Adulthood_Total,nbd_tot
0,72.0,12.0,6.0,12.0,0.0,23.0,7.0,14,9.0,19.0,13,18,15,31.3636
1,89.0,16.0,11.0,11.0,19.0,35.0,17.0,30,41.0,31.0,20,17,18,71.2311
2,82.0,16.0,8.0,9.0,8.0,35.0,11.0,33,14.0,18.0,11,12,14,64.9621
3,48.0,6.0,6.0,5.0,6.0,23.0,7.0,4,9.0,17.0,12,14,17,32.0833
4,43.0,5.0,4.0,4.0,1.0,23.0,7.0,5,1.0,3.0,11,11,13,31.9318


In [14]:
neuropsych.describe()

,bistot,abis_attention,abis_motor,abis_nonplanning,bhstot,bdhi_total,cnstot,bditot,baitot,pcltot,BGLHA_Childhood_Total,BGLHA_Adolescence_Total,BGLHA_Adulthood_Total,nbd_tot
count,236.000000,236.000000,236.00000,236.000000,236.000000,236.00000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000
mean,61.931624,10.136752,7.34188,8.538462,3.000000,26.25974,12.038462,10.067797,7.884615,14.888412,12.944915,13.779661,15.521186,42.829579
std,14.042626,3.701422,2.40093,3.098492,4.237623,14.06405,4.293812,9.901841,8.824021,15.671780,3.418525,4.137294,4.625965,13.800428
min,33.000000,5.000000,4.00000,4.000000,0.000000,3.00000,7.000000,0.000000,0.000000,0.000000,11.000000,11.000000,11.000000,27.197000
25%,51.000000,7.000000,5.00000,6.000000,0.000000,16.00000,8.000000,2.000000,1.000000,3.000000,11.000000,11.000000,12.000000,31.363600
50%,61.000000,10.000000,7.00000,8.000000,1.000000,24.00000,11.000000,8.000000,5.000000,10.000000,11.500000,12.000000,14.000000,38.579550
75%,72.000000,13.000000,9.00000,11.000000,4.000000,35.00000,15.000000,16.000000,12.000000,21.250000,13.000000,15.000000,18.000000,50.416675
max,107.000000,20.000000,15.00000,16.000000,19.000000,65.00000,29.000000,42.000000,42.000000,75.000000,32.000000,34.000000,37.000000,87.197000


In [15]:
neuropsych.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bistot                   236 non-null    float64
 1   abis_attention           236 non-null    float64
 2   abis_motor               236 non-null    float64
 3   abis_nonplanning         236 non-null    float64
 4   bhstot                   236 non-null    float64
 5   bdhi_total               236 non-null    float64
 6   cnstot                   236 non-null    float64
 7   bditot                   236 non-null    int64  
 8   baitot                   236 non-null    float64
 9   pcltot                   236 non-null    float64
 10  BGLHA_Childhood_Total    236 non-null    int64  
 11  BGLHA_Adolescence_Total  236 non-null    int64  
 12  BGLHA_Adulthood_Total    236 non-null    int64  
 13  nbd_tot                  236 non-null    float64
dtypes: float64(10), int64(4)
m

In [16]:
import os
main_path = r'Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers'
folder_name = 'blood_biomk1_to_neuropsych'  
save_path = os.path.join(main_path, folder_name)
os.makedirs(save_path, exist_ok=True)


In [17]:
categories_file_path = r"Z:\Active-Diagnose_CTE\Fargol_Analysis\Neuropsychiatric_from_Biomarkers\categories.csv"
categories_df = pd.read_csv(categories_file_path)
new_column = categories_df['checkin_bin']
neuropsych['category'] = new_column
print(neuropsych.head())

bloodbiomk2['category'] = new_column
print(bloodbiomk2.head())


   bistot  abis_attention  abis_motor  abis_nonplanning  bhstot  bdhi_total  \
0    72.0            12.0         6.0              12.0     0.0        23.0   
1    89.0            16.0        11.0              11.0    19.0        35.0   
2    82.0            16.0         8.0               9.0     8.0        35.0   
3    48.0             6.0         6.0               5.0     6.0        23.0   
4    43.0             5.0         4.0               4.0     1.0        23.0   

   cnstot  bditot  baitot  pcltot  BGLHA_Childhood_Total  \
0     7.0      14     9.0    19.0                     13   
1    17.0      30    41.0    31.0                     20   
2    11.0      33    14.0    18.0                     11   
3     7.0       4     9.0    17.0                     12   
4     7.0       5     1.0     3.0                     11   

   BGLHA_Adolescence_Total  BGLHA_Adulthood_Total  nbd_tot  category  
0                       18                     15  31.3636         2  
1                     

In [18]:
# csfbiomk2['category'] = neuropsych['category']

# correlation_matrix = csfbiomk2.corr()

# plt.figure(figsize=(12, 8))
# sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True)
# plt.title('Correlation Heatmap of Features')
# plt.show()

# # Alternatively, scatter plots for each feature vs y (checkin_bin)
# for feature in csfbiomk2.columns[:-1]:  # Exclude the label column
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='category', y=feature, data=csfbiomk2)
#     plt.title(f'Boxplot of {feature} by category')
#     plt.xlabel('category')
#     plt.ylabel(feature)
#     plt.show()


In [19]:
csfbiomk2.copy()

,Qalb,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,c_IL_17D,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,7.852013,30.721000,7.79000,0.089800,1.090000,16.200000,9.240000,0.04770,0.646000,384.000000,...,0.789000,1.070000,0.653000,2.400000,0.159000,2.220000,2.090000,2.549289,336.278146,1.000000
1,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
2,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
3,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
4,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,3.776207,12.716000,12.40000,0.062900,3.670000,11.100000,13.800000,0.03790,0.877000,375.000000,...,2.180000,5.340000,0.969000,6.760000,0.099900,5.500000,1.450000,4.070000,201.000000,0.000000
232,7.384902,24.221000,1.14000,0.240000,0.524000,15.600000,1.520000,0.02720,0.213000,536.000000,...,0.106000,6.240000,0.718000,7.940000,0.172000,0.474000,1.540000,2.070000,258.000000,0.000000
233,4.568572,17.229000,19.20000,0.298000,0.624000,6.560000,10.300000,0.01100,0.336000,328.000000,...,0.968000,4.400000,0.551000,6.540000,0.025500,0.260000,1.360000,4.110000,156.000000,0.000000
234,11.101832,32.172000,0.67000,0.099800,2.600000,17.500000,0.660000,0.03410,1.670000,430.000000,...,1.760000,2.860000,0.861000,2.890000,0.096800,0.996000,2.680000,1.150000,276.000000,0.000000



# 95th percentile for both analysis and plotting

In [20]:
for feature in csfbiomk2.columns[:-1]:  # Assuming 'category' is the last column
    upper_limit = csfbiomk2[feature].quantile(0.95)  # Cap at 95th percentile
    lower_limit = csfbiomk2[feature].quantile(0.05)  # Floor at 5th percentile
    csfbiomk2[feature] = np.where(csfbiomk2[feature] > upper_limit, upper_limit, csfbiomk2[feature])
    csfbiomk2[feature] = np.where(csfbiomk2[feature] < lower_limit, lower_limit, csfbiomk2[feature])

    

# Initial Step for Trainings

In [21]:
X = csfbiomk2.copy()

In [22]:
X

,Qalb,c_alb,c_IL_17A_F_cv,c_IL_17A_F,c_IL_17B_cv,c_IL_17B,c_IL_17C_cv,c_IL_17C,c_IL_17D_cv,c_IL_17D,...,c_IL_5,c_IL_7_cv,c_IL_7,c_TNF_beta_cv,c_TNF_beta,c_VEGF_A_cv,c_VEGF_A,c_asyn_cv,c_asyn,dxcte_asyn_c_flag_dv
0,7.852013,30.721000,7.79000,0.089800,1.090000,16.200000,9.240000,0.04770,0.646000,384.000000,...,0.789000,1.070000,0.653000,2.400000,0.159000,2.220000,2.090000,2.549289,336.278146,1.000000
1,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
2,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
3,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
4,7.564584,25.774017,6.55229,0.188555,3.466409,13.910568,7.537295,0.04312,3.764716,396.193182,...,1.153024,3.541709,0.796944,4.822307,0.108486,3.506305,1.906564,2.549289,336.278146,0.142045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,4.213896,13.767250,12.40000,0.062900,3.670000,11.100000,13.800000,0.03790,0.877000,375.000000,...,1.950000,5.340000,0.969000,6.760000,0.099900,5.500000,1.450000,4.070000,201.000000,0.000000
232,7.384902,24.221000,1.14000,0.240000,0.524000,15.600000,1.520000,0.02720,0.306750,530.750000,...,0.159250,6.240000,0.718000,7.940000,0.172000,0.474000,1.540000,2.070000,258.000000,0.000000
233,4.568572,17.229000,16.10000,0.298000,0.624000,6.560000,10.300000,0.01390,0.336000,328.000000,...,0.968000,4.400000,0.551000,6.540000,0.025500,0.260000,1.360000,4.110000,184.750000,0.000000
234,11.101832,32.172000,1.08000,0.099800,2.600000,17.500000,1.132500,0.03410,1.670000,430.000000,...,1.760000,2.860000,0.861000,2.890000,0.096800,0.996000,2.680000,1.150000,276.000000,0.000000


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# got error due to non numeric values, so removing them:

# Assume csfbiomk2 is your DataFrame loaded with various types of data
X = csfbiomk2.copy()

# Select only numeric columns from the DataFrame
X_numeric = X.select_dtypes(include=[np.number])
X = X_numeric

# Now X_numeric contains only the numeric columns of the original DataFrame
# Proceed with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing sets
X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)


models = {}
predictions = {}


X_train, X_test = train_test_split(X_scaled, test_size=0.3, random_state=42)

In [24]:
# Define the columns you want to select for modeling

# selected_columns = ['bistot','abis_attention','abis_motor','abis_nonplanning','bhstot','bdhi_total', 'cnstot', 'bditot', 'baitot', 'pcltot', 
# 'BGLHA_Childhood_Total', 'BGLHA_Adolescence_Total', 'BGLHA_Adulthood_Total', 'nbd_tot' ]

selected_columns = ['bditot']


In [25]:
selected_columns

['bditot']

In [26]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Initialize an empty list to store results
results = []
feature_importance_rankings = {}
model_name = "Random Forest"
selected_columns_str = "_".join(selected_columns)

# First, calculate feature importances for each target column and save rankings
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Get feature importance and rank features
    feature_importance = pd.Series(model.feature_importances_, index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Save performance metrics with all features as a baseline
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")


# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to feature_importance_rankings.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")



Feature importance rankings saved to feature_importance_rankings.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [27]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Multiple Linear Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for linear regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [28]:
import pandas as pd
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gradient Boosting Machines"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = LGBMRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000531 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4295
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 117
[LightGBM] [Info] Start training from score 10.042424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000027 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 294
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 8
[LightGBM] [Info] Start training from score 10.042424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000073 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 733
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 20
[LightGBM] [Info] Start training from score 10.042424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1210
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 33
[LightGBM] [Info] Start training from score 10.042424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3995
[LightGBM] [Info] Number of data points in the train set: 165, number of used features: 109
[LightGBM] [Info] Start training from score 10.042424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [29]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "XGBoost"
selected_columns_str = "_".join(selected_columns)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train full model with all features as a baseline
    model = xgb.XGBRegressor(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = xgb.XGBRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [30]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Support Vector Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for SVR
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train SVR model with all features as a baseline
    model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = SVR(kernel='rbf', C=0.2, epsilon=0.01)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [31]:
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "ANN"
selected_columns_str = "_".join(selected_columns)

# Standardize features for ANN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train ANN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = Sequential([
        Dense(128, activation='relu', input_dim=X_train_selected.shape[1]),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_selected, y_train, epochs=100, batch_size=10, verbose=0)
    
    y_pred = model.predict(X_test_selected).flatten()
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Scale selected features for ANN
        X_train_selected_scaled = scaler.fit_transform(X_train_selected)
        X_test_selected_scaled = scaler.transform(X_test_selected)
        
        # Train and evaluate ANN model with selected features
        model = Sequential([
            Dense(128, activation='relu', input_dim=X_train_selected_scaled.shape[1]),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X_train_selected_scaled, y_train, epochs=100, batch_size=10, verbose=0)
        
        y_pred = model.predict(X_test_selected_scaled).flatten()
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


3/3 [==============================] - 0s 1ms/step


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


3/3 [==============================] - 0s 997us/step
Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [32]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "KNeighbors Regressor"
selected_columns_str = "_".join(selected_columns)

# Standardize features for KNN
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature selection and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train KNN model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    model = KNeighborsRegressor(n_neighbors=5)
    model.fit(X_train_selected, y_train)
    
    y_pred = model.predict(X_test_selected)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate KNN model with selected features
        model = KNeighborsRegressor(n_neighbors=5)
        model.fit(X_train_selected, y_train)
        
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [33]:
import pandas as pd
from sklearn.ensemble import StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Stacking Regressor"
selected_columns_str = "_".join(selected_columns)

# Define the base models and final estimator for stacking
base_models = [
    ('knn', KNeighborsRegressor(n_neighbors=13, leaf_size=23, p=1, weights='uniform')),
    ('svr', SVR(kernel='rbf', C=0.2, epsilon=0.01))
]
final_estimator = Ridge()
stack_model = StackingRegressor(estimators=base_models, final_estimator=final_estimator, cv=5)

# Feature selection and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train stacking model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    stack_model.fit(X_train_selected, y_train)
    y_pred = stack_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate stacking model with selected features
        stack_model.fit(X_train_selected, y_train)
        y_pred = stack_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [34]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [35]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Bagging with SVR"
selected_columns_str = "_".join(selected_columns)

# Define the base model for Bagging
base_model = SVR(kernel='rbf', C=0.2, epsilon=0.01)

# Feature selection and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train Bagging model with all features as a baseline
    X_train_selected = selector.transform(X_train)
    X_test_selected = selector.transform(X_test)
    
    bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
    bagging_model.fit(X_train_selected, y_train)
    y_pred = bagging_model.predict(X_test_selected)
    
    # Evaluate and store baseline performance metrics with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate Bagging model with selected features
        bagging_model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=42, n_jobs=-1)
        bagging_model.fit(X_train_selected, y_train)
        y_pred = bagging_model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:1

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and w

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [36]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Lasso Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Lasso regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Lasso Regression Model with L1 regularization for feature selection
    model = Lasso(alpha=0.1)
    model.fit(X_train, y_train)
    
    # Select features based on non-zero coefficients
    selector = SelectFromModel(model, prefit=True)
    selected_features = X_train.columns[selector.get_support()]
    
    # Feature importance ranking based on absolute Lasso coefficients
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features

    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Lasso(alpha=0.1)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [37]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel

# Initialize lists and dictionaries to store results and feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Ridge Regression"
selected_columns_str = "_".join(selected_columns)

# Standardize features for Ridge regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Ridge Regression Model
    model = Ridge(alpha=1.0)
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Evaluate baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = Ridge(alpha=1.0)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [38]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Initialize lists and dictionaries for storing results and feature rankings
results = []
feature_importance_rankings = {}
model_name = "Bayesian Ridge"
selected_columns_str = "_".join(selected_columns)

# Standardize features for consistency
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Feature ranking and baseline model training
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Train full model to get feature importances
    model = BayesianRidge()
    model.fit(X_train, y_train)
    
    # Use coefficients for feature importance ranking
    feature_importance = pd.Series(abs(model.coef_), index=X_train.columns)
    ranked_features = feature_importance.sort_values(ascending=False)
    feature_importance_rankings[column] = ranked_features
    
    # Baseline performance with all features
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select top `n_features` based on ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = BayesianRidge()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx


In [39]:
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

# Initialize an empty list to store results and a dictionary for feature importance rankings
results = []
feature_importance_rankings = {}
model_name = "Gaussian Process Regression"
selected_columns_str = "_".join(selected_columns)

# Define the kernel
kernel = RBF(1.0)

# Standardize features for Gaussian Process Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# First, calculate feature importances using SelectKBest for each target column and save rankings
for column in selected_columns:
    y = neuropsych[column]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
    
    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_regression, k="all")
    selector.fit(X_train, y_train)
    feature_scores = pd.Series(selector.scores_, index=X_train.columns)
    ranked_features = feature_scores.sort_values(ascending=False)
    
    # Save the rankings in a dictionary
    feature_importance_rankings[column] = ranked_features
    
    # Train the full Gaussian Process model as a baseline
    model = GaussianProcessRegressor(kernel=kernel, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Evaluate and store baseline performance with all features
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        "Model": model_name,
        "Column": column,
        "Top Features": "All",
        "Mean Squared Error": mse,
        "Mean Absolute Error": mae,
        "R^2 Score": r2
    })

# Save feature importance rankings to an Excel file
with pd.ExcelWriter(f"Feature_Importance_csfbiomk2_rank_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    for column, ranking in feature_importance_rankings.items():
        ranking_sorted = ranking.sort_values(ascending=False)
        ranking_sorted.to_excel(writer, sheet_name=f"{model_name}_{column}")

# Test different numbers of top features and evaluate model performance
max_features_to_test = X.shape[1]  # Test from 1 up to the total number of features

for n_features in range(1, max_features_to_test + 1):
    for column in selected_columns:
        y = neuropsych[column]
        X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.3, random_state=42)
        
        # Select the top `n_features` based on initial ranking
        top_features = feature_importance_rankings[column].nlargest(n_features).index
        X_train_selected = X_train[top_features]
        X_test_selected = X_test[top_features]
        
        # Train and evaluate model with selected features
        model = GaussianProcessRegressor(kernel=kernel, random_state=42)
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Append results for each number of top features
        results.append({
            "Model": model_name,
            "Column": column,
            "Top Features": n_features,
            "Mean Squared Error": mse,
            "Mean Absolute Error": mae,
            "R^2 Score": r2
        })

# Save all results to an Excel file
results_df = pd.DataFrame(results)
with pd.ExcelWriter(f"performance_csfbiomk2_w_top_features_{selected_columns_str}.xlsx", mode="a", if_sheet_exists="new") as writer:
    results_df.to_excel(writer, index=False, sheet_name=model_name)

print("Feature importance rankings saved to Feature_Importance_rank.xlsx")
print("Performance metrics with varying top features saved to performance_with_top_features.xlsx")


C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
C:\Users\fargor\AppData\Local\anaconda3\lib\site-packages\sklearn\gaussian_proc

Feature importance rankings saved to Feature_Importance_rank.xlsx
Performance metrics with varying top features saved to performance_with_top_features.xlsx
